In [ ]:
!pip install firebase-admin
!pip install google-cloud-storage
!pip install python-dotenv
!pip install pandas


In [55]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

# Firebase Init

In [90]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
}

In [91]:
import firebase_admin
from firebase_admin import credentials

# Delete any existing Firebase apps to avoid conflicts
try:
    for app in firebase_admin._apps.values():
        firebase_admin.delete_app(app)
    print("Deleted existing Firebase apps")
except:
    pass

# Initialize Firebase with the correct configuration
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred, {
    'storageBucket': 'coffee-shop-app-51c4f.firebasestorage.app',
    'databaseURL': 'https://coffee-shop-app-51c4f-default-rtdb.firebaseio.com/'
})

print("Firebase initialized successfully")

Firebase initialized successfully


In [92]:
bucket = storage.bucket()
print(f"Using bucket: {bucket.name}")
print(f"Expected bucket: coffee-shop-app-51c4f.firebasestorage.app")

Using bucket: coffee-shop-app-51c4f.firebasestorage.app
Expected bucket: coffee-shop-app-51c4f.firebasestorage.app


# Upload Data

In [79]:
image_folder_path = './products/images/'

In [93]:
products_collection = db.reference('products')

In [81]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [ ]:
def upload_image(bucket, image_path):
    print(f"Bucket name: {bucket.name}")
    print(f"Image path: {image_path}")
    print(f"File exists: {os.path.exists(image_path)}")
    
    image_name = image_path.split('/')[-1]
    blob_path = f'product_images/{image_name}'
    print(f"Blob path: {blob_path}")
    
    blob = bucket.blob(blob_path)
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [95]:
for index, row in df.iterrows():
    print(f"Processing {index}: {row['name']}")
    
    image_path = os.path.join(image_folder_path, row['image_path'])
    
    # Check if image file exists before uploading
    if not os.path.exists(image_path):
        print(f"  Warning: Image file not found: {image_path}")
        continue
    
    try:
        image_url = upload_image(bucket, image_path)
        product_data = row.to_dict()
        product_data.pop('image_path')
        product_data['image_url'] = image_url
        
        # Add to Firebase Realtime Database
        products_collection.push(product_data)
        print(f"  ✓ Successfully uploaded: {row['name']}")
        
    except Exception as e:
        print(f"  ✗ Error uploading {row['name']}: {e}")
        continue

print("Upload process completed!")

Processing 0: Cappuccino
  ✓ Successfully uploaded: Cappuccino
Processing 1: Jumbo Savory Scone
  ✓ Successfully uploaded: Cappuccino
Processing 1: Jumbo Savory Scone
  ✓ Successfully uploaded: Jumbo Savory Scone
Processing 2: Latte
  ✓ Successfully uploaded: Jumbo Savory Scone
Processing 2: Latte
  ✓ Successfully uploaded: Latte
Processing 3: Chocolate Chip Biscotti
  ✓ Successfully uploaded: Latte
Processing 3: Chocolate Chip Biscotti
  ✓ Successfully uploaded: Chocolate Chip Biscotti
Processing 4: Espresso shot
  ✓ Successfully uploaded: Chocolate Chip Biscotti
Processing 4: Espresso shot
  ✓ Successfully uploaded: Espresso shot
Processing 5: Hazelnut Biscotti
  ✓ Successfully uploaded: Espresso shot
Processing 5: Hazelnut Biscotti
  ✓ Successfully uploaded: Hazelnut Biscotti
Processing 6: Chocolate Croissant
  ✓ Successfully uploaded: Hazelnut Biscotti
Processing 6: Chocolate Croissant
  ✓ Successfully uploaded: Chocolate Croissant
Processing 7: Dark chocolate
  ✓ Successfully uplo

In [94]:
# Test bucket access first
print(f"Bucket name: {bucket.name}")
print(f"Project: {bucket.client.project}")

# Test if we can list existing files
try:
    blobs = list(bucket.list_blobs(max_results=5))
    print(f"Found {len(blobs)} existing files in bucket")
    for blob in blobs:
        print(f"  - {blob.name}")
except Exception as e:
    print(f"Error listing files: {e}")

# Check if the local products.jsonl file exists and can be read
print(f"Products file exists: {os.path.exists('products/products.jsonl')}")
print(f"Images folder exists: {os.path.exists('products/images/')}")

# List some image files
if os.path.exists('products/images/'):
    image_files = os.listdir('products/images/')[:5]
    print(f"Sample image files: {image_files}")
else:
    print("Images folder not found!")

Bucket name: coffee-shop-app-51c4f.firebasestorage.app
Project: coffee-shop-app-51c4f
Found 5 existing files in bucket
  - product_images/
  - product_images/Chocolate_Croissant.jpg
  - product_images/Chocolate_syrup.jpg
  - product_images/Cranberry_Scone.jpg
  - product_images/Croissant.jpg
Products file exists: True
Images folder exists: True
Sample image files: ['almond_croissant.jpg', 'cappuccino.jpg', 'caramel_syrup.jpg', 'Chocolate_Croissant.jpg', 'Chocolate_syrup.jpg']
